In [93]:
# imports a library 'pandas', names it as 'pd'

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV


# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [94]:
read_in_file = pd.read_csv('C:/Users/smnio/Desktop/Projects/local_only/appt1.csv')#, nrows=1000000)

In [95]:
read_in_file = read_in_file.dropna()
df= read_in_file.copy()
df.head()

,﻿app_month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,eth_hispanic,lang_Eng,lang_Chi,lang_Spa,lang_Other,male,female,income,depression,preg
0,5,14,2,1,0,0,1,0,0,1.000000,...,0,1,0,0,0,0,1,83622,0,0
1,7,9,3,1,0,0,1,0,0,0.785714,...,0,1,0,0,0,0,1,83622,0,0
2,10,13,6,1,0,0,1,0,0,0.800000,...,0,1,0,0,0,0,1,83622,0,0
3,11,10,5,1,0,0,0,0,1,0.750000,...,0,1,0,0,0,0,1,83622,0,0
4,11,11,5,1,0,0,0,0,1,0.777777,...,0,1,0,0,0,0,1,83622,0,0


In [96]:
cols = ['month']
for i in range(1,len(df.columns)):
    cols.append(df.columns[i])
df.columns=cols

df = pd.concat([df, pd.get_dummies(df['month'], prefix='month')], axis=1)

In [97]:
df = pd.concat([df, pd.get_dummies(df['weekday'], prefix='wkday')], axis=1)

In [98]:
df = pd.concat([df, pd.get_dummies(df['appt_begintime'], prefix='hour')], axis=1)

In [99]:
df.income = df.income.round(-4)
df = pd.concat([df, pd.get_dummies(df['income'], prefix='inc')], axis=1)

In [100]:
#df['income'] = df.income.map(lambda x: df['income'].median() if (x > 0))

In [101]:
#break out ages into discrete groups

df['age_65_and_over'] = 0
df['age_under_5'] = 0
df['age_5_17'] = 0
df['age_18_29'] = 0
df['age_30_44'] = 0
df['age_45_64'] = 0


df['age_65_and_over'] = df.age.map(lambda x: 1 if (x > 64) else 0)
df['age_under_5'] = df.age.map(lambda x: 1 if (x < 5) else 0)
df['age_5_17'] = df.age.map(lambda x: 1 if (x > 4 and x <18 ) else 0)
df['age_18_29'] = df.age.map(lambda x: 1 if (x > 17 and x <30 ) else 0)
df['age_30_44'] = df.age.map(lambda x: 1 if (x > 29 and x <45 ) else 0)
df['age_45_64'] = df.age.map(lambda x: 1 if (x > 44 and x <65 ) else 0)

df.head()

,month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,inc_190000,inc_200000,inc_210000,inc_220000,age_65_and_over,age_under_5,age_5_17,age_18_29,age_30_44,age_45_64
0,5,14,2,1,0,0,1,0,0,1.000000,...,0,0,0,0,0,0,0,0,0,1
1,7,9,3,1,0,0,1,0,0,0.785714,...,0,0,0,0,1,0,0,0,0,0
2,10,13,6,1,0,0,1,0,0,0.800000,...,0,0,0,0,0,0,0,0,0,1
3,11,10,5,1,0,0,0,0,1,0.750000,...,0,0,0,0,1,0,0,0,0,0
4,11,11,5,1,0,0,0,0,1,0.777777,...,0,0,0,0,0,0,0,0,0,1


In [102]:
columnNames=df.columns

In [103]:
from sklearn import preprocessing

xi = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
xi_scaled = min_max_scaler.fit_transform(xi)
df = pd.DataFrame(xi_scaled)


df.columns=columnNames



In [104]:
df.corr()

,month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,inc_190000,inc_200000,inc_210000,inc_220000,age_65_and_over,age_under_5,age_5_17,age_18_29,age_30_44,age_45_64
month,1.000000,0.000238,-0.031025,0.026211,-0.022849,-0.012780,0.023939,-0.020065,-0.012180,0.021746,...,-0.001643,-0.000821,-0.000821,-0.000159,0.003769,0.007170,0.009051,0.004124,-0.010995,-0.003722
appt_begintime,0.000238,1.000000,-0.064368,-0.026077,0.009018,0.023271,-0.042826,0.018559,0.035599,-0.055801,...,-0.001726,0.000884,0.000716,0.002678,-0.072501,-0.001393,0.107106,0.050495,0.011967,-0.028711
weekday,-0.031025,-0.064368,1.000000,-0.000404,-0.001020,0.001253,-0.010700,0.005722,0.008030,-0.010888,...,-0.002418,0.000421,0.000726,-0.000762,-0.011104,0.001626,0.024148,0.007261,-0.005970,-0.001820
kept,0.026211,-0.026077,-0.000404,1.000000,-0.524081,-0.755158,0.117257,-0.071272,-0.081175,0.214077,...,0.000531,-0.000958,0.001034,0.000767,0.048259,0.033102,0.006574,-0.029676,-0.015837,-0.025246
rescheduled,-0.022849,0.009018,-0.001020,-0.524081,1.000000,-0.162542,-0.066890,0.120697,-0.017450,-0.131592,...,0.002079,0.000435,-0.001064,-0.000402,-0.023776,-0.012441,-0.000497,-0.008199,0.002772,0.029587
cancelled,-0.012780,0.023271,0.001253,-0.755158,-0.162542,1.000000,-0.084362,-0.010331,0.107477,-0.146730,...,-0.002216,0.000775,-0.000379,-0.000579,-0.037610,-0.028775,-0.007233,0.040692,0.016215,0.006475
last_kept,0.023939,-0.042826,-0.010700,0.117257,-0.066890,-0.084362,1.000000,-0.543455,-0.743562,0.530223,...,-0.000151,-0.000040,0.000968,0.000747,0.051424,0.029428,0.005696,-0.028600,-0.016605,-0.026615
last_rescheduled,-0.020065,0.018559,0.005722,-0.071272,0.120697,-0.010331,-0.543455,1.000000,-0.157212,-0.282193,...,0.001227,0.000415,-0.001074,-0.000406,-0.024845,-0.012613,-0.001825,-0.006793,0.003590,0.029628
last_cancelled,-0.012180,0.035599,0.008030,-0.081175,-0.017450,0.107477,-0.743562,-0.157212,1.000000,-0.399001,...,-0.000800,-0.000284,-0.000284,-0.000555,-0.040708,-0.024573,-0.005248,0.039058,0.016675,0.007710
kept_perc,0.021746,-0.055801,-0.010888,0.214077,-0.131592,-0.146730,0.530223,-0.282193,-0.399001,1.000000,...,0.001517,0.002598,0.000913,0.001329,0.097812,0.056406,0.011508,-0.048304,-0.036909,-0.050714


In [105]:
#df.corr()['kept'].order(ascending=False)

In [106]:
#k=corrVals[corrVals.ix[:1]>.001]
#print(k)

In [107]:
corr_series = df.corr()['kept'].order(ascending=False)


print(corr_series)

kept                1.000000
kept_perc           0.214077
last_kept           0.117257
income              0.067245
lang_Eng            0.056029
age_65_and_over     0.048259
age_under_5         0.033102
inc_100000          0.027178
age                 0.026446
month               0.026211
male                0.025262
inc_120000          0.023937
eth_refused         0.020843
inc_90000           0.019538
inc_110000          0.018758
hour_4              0.017570
hour_5              0.017464
hour_2              0.016739
hour_3              0.016595
hour_1              0.016274
hour_12             0.015214
hour_6              0.015038
inc_130000          0.013849
month_10            0.012485
inc_150000          0.011620
inc_170000          0.010383
lang_Other          0.009674
month_11            0.009144
inc_70000           0.009049
month_6             0.008862
                      ...   
wkday_1            -0.006189
hour_15            -0.007738
lang_Spa           -0.007946
hour_23       

In [108]:
selected_columns = corr_series[((corr_series > .02 ) | (corr_series < -.02))].index

selected_columns = selected_columns[((selected_columns != 'kept' ) & (selected_columns != 'cancelled') & (selected_columns != 'rescheduled'))]

print(selected_columns)

Index([u'kept_perc', u'last_kept', u'income', u'lang_Eng', u'age_65_and_over',
       u'age_under_5', u'inc_100000', u'age', u'month', u'male', u'inc_120000',
       u'eth_refused', u'hour_20', u'age_45_64', u'female', u'appt_begintime',
       u'depression', u'age_18_29', u'inc_50000', u'inc_30000', u'month_2',
       u'lang_Chi', u'last_rescheduled', u'last_cancelled', u'resch_perc',
       u'can_perc'],
      dtype='object')


In [109]:

X = df.ix[:, selected_columns] 

y = df.ix[:, df.columns == 'kept']




In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [111]:


#np.all(np.isfinite(mat))

In [120]:
#supervised classification
from sklearn.neighbors import NearestNeighbors

lr=LogisticRegression()

#lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

AttributeError: 'NearestNeighbors' object has no attribute 'score'

In [113]:
#0.71724745080587426

In [114]:
read_in_file = read_in_file.dropna()



y_pred = lr.predict(X)



data = pd.DataFrame({"Actual":read_in_file.kept, "Predicted":y_pred.tolist()})



In [115]:
#get calculated coeffcicients
lr.coef_

array([[ 0.56920632,  0.15664835,  0.72763214, -0.03664909,  0.17387664,
         0.38522381,  0.025626  , -0.21029382,  0.07927446,  0.22708217,
        -0.02117388,  0.06646523, -1.19828382,  0.00523875,  0.17122937,
        -0.22302105, -0.09529189, -0.11386599, -0.12338746, -0.08222906,
        -0.21497296, -0.21381354,  0.17450548,  0.14080017, -1.39907963,
        -1.07236483]])

In [116]:
import csv

data_top=data.head(10000)

data_top.to_csv("predictions.csv")



IOError: [Errno 13] Permission denied: 'predictions.csv'